In [1]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn as nn
from torchvision import datasets,transforms
import torch.nn.functional as F
import sys
sys.path.append("../code/")
from dataloader import APPLIANCE_ORDER, get_train_test
from sklearn.metrics import mean_absolute_error
import os
import pandas as pd
import sys
import numpy as np
import matplotlib.pyplot as plt
sys.path.append('../code/')
%matplotlib inline
import itertools
from pathlib import Path

In [2]:
def onoff_error(pred, gt, threshold):
    abs_error = np.abs(pred-gt)
    error = [x for x in abs_error.reshape(1, -1).tolist()[0] if x >= threshold]
    
    return np.mean(error)

In [3]:
dataset = 5
gid = 0
srv = 1
c = 0
cmd = {}
cnn_tree_valid_pred = {}
num_iterations = 20000
lr = 0.01
p = 0
for fold_num in range(5):
    cnn_tree_valid_pred[fold_num] = {}
    for lr in [0.01]:
        cnn_tree_valid_pred[fold_num][lr] = {}
        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw', 'residual'])):
            print(fold_num, lr, order)
#             if order[0] == 'hvac':
#                 continue
            
            cnn_tree_valid_pred[fold_num][lr][order] = {}


            o = "\', \'".join(str(x) for x in order)
            directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num, lr)
            filename = "valid-pred-[\'{}\'].npy".format(o)

            full_path = directory + filename
            my_file = Path(full_path)
            if not my_file.exists():
                o = " ".join(str(x) for x in order)
                line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 3 {} {} 0 {} {} &".format(gid, lr, num_iterations, fold_num, o))
#                 print(line)

                c += 1
                if c%1 == 0:
                    gid+=1
                if gid == 4:
                    gid = 0
                    srv += 1
            else:
                k = np.load(full_path).item()
                for it in range(1000, 20001, 1000):
                    cnn_tree_valid_pred[fold_num][lr][order][it] = {}
                    for idx, appliance in enumerate(order):
                        cnn_tree_valid_pred[fold_num][lr][order][it][appliance] = k[it][idx]

print(c)                               

0 0.01 ('hvac', 'fridge', 'dr', 'dw', 'mw', 'residual')
0 0.01 ('hvac', 'fridge', 'dr', 'dw', 'residual', 'mw')
0 0.01 ('hvac', 'fridge', 'dr', 'mw', 'dw', 'residual')
0 0.01 ('hvac', 'fridge', 'dr', 'mw', 'residual', 'dw')
0 0.01 ('hvac', 'fridge', 'dr', 'residual', 'dw', 'mw')
0 0.01 ('hvac', 'fridge', 'dr', 'residual', 'mw', 'dw')
0 0.01 ('hvac', 'fridge', 'dw', 'dr', 'mw', 'residual')
0 0.01 ('hvac', 'fridge', 'dw', 'dr', 'residual', 'mw')
0 0.01 ('hvac', 'fridge', 'dw', 'mw', 'dr', 'residual')
0 0.01 ('hvac', 'fridge', 'dw', 'mw', 'residual', 'dr')
0 0.01 ('hvac', 'fridge', 'dw', 'residual', 'dr', 'mw')
0 0.01 ('hvac', 'fridge', 'dw', 'residual', 'mw', 'dr')
0 0.01 ('hvac', 'fridge', 'mw', 'dr', 'dw', 'residual')
0 0.01 ('hvac', 'fridge', 'mw', 'dr', 'residual', 'dw')
0 0.01 ('hvac', 'fridge', 'mw', 'dw', 'dr', 'residual')
0 0.01 ('hvac', 'fridge', 'mw', 'dw', 'residual', 'dr')
0 0.01 ('hvac', 'fridge', 'mw', 'residual', 'dr', 'dw')
0 0.01 ('hvac', 'fridge', 'mw', 'residual', 'dw'

KeyboardInterrupt: 

In [10]:
tensor = np.load("../2015-5appliances.numpy.npy")
dr = tensor[:, 3]

# calculate gt
test_gt = {}
valid_gt = {}
for fold_num in range(5):
    test_gt[fold_num] = {}
    valid_gt[fold_num] = {}
    train, test = get_train_test(1, 5, fold_num)
    valid = train[int(0.8*len(train)):].copy()
    for idx, appliance in enumerate(APPLIANCE_ORDER[1:-1]):
        test_gt[fold_num][appliance] = test[:, idx+1]
        valid_gt[fold_num][appliance] = valid[:, idx+1]
        
threshold = {}
for appliance in ['dr', 'dw', 'mw']:
    sample_list = []
    for fold_num in range(5):
        sample_list = np.append(sample_list, [x for x in test_gt[fold_num][appliance].reshape(1, -1).tolist()[0] if x > 5])
    mean = np.mean(sample_list)
    print(appliance, mean)
    threshold[appliance] = 0.1*mean

dr 725.849570636
dw 285.780713239
mw 31.7082756317


In [23]:
cnn_tree_best_param = {}
for fold_num in range(5):
    cnn_tree_best_param[fold_num] = {}
    min_error = np.inf
    for lr in [0.01]:
        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw', 'residual'])):

            for it in range(1000, 20001, 1000):
                error = 0
                try:
                    for idx, appliance in enumerate(order):
                        if appliance == 'residual':
                            continue
                        if appliance in ['hvac', 'fridge']:
                            error += mean_absolute_error(cnn_tree_valid_pred[fold_num][lr][order][it][appliance].reshape(-1, 24),
                                                        valid_gt[fold_num][appliance].reshape(-1, 24))
                        else:
                            error += onoff_error(cnn_tree_valid_pred[fold_num][lr][order][it][appliance].reshape(-1, 24),
                                                        valid_gt[fold_num][appliance].reshape(-1, 24), threshold[appliance])
                    if error < min_error:
                        min_error = error
                        cnn_tree_best_param[fold_num]['lr'] = lr
                        cnn_tree_best_param[fold_num]['order'] = order
                        cnn_tree_best_param[fold_num]['iters'] = it
                except:
                    continue
                
                    

In [24]:
best_cnn_tree = cnn_tree_best_param

In [25]:
# get prediction for cnn tree
cnn_tree_pred = {}
for fold_num in range(5):
    cnn_tree_pred[fold_num] = {}
        
    lr = best_cnn_tree[fold_num]['lr']
    iters = best_cnn_tree[fold_num]['iters']
    order = best_cnn_tree[fold_num]['order']

    o = "\', \'".join(str(x) for x in order)
    directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num,lr)
    filename = "test-pred-[\'{}\'].npy".format(o)

    full_path = directory + filename
    cnn_pred = np.asarray(np.load(full_path).item()[iters])
    
    for idx, appliance in enumerate(order):
        cnn_tree_pred[fold_num][appliance] = cnn_pred[idx].reshape(-1, 1, 112,24)
    

In [26]:
def calculate_error(pred, gt, threshold):
    error = {}
    overall = {}
    num_homes = {}
    
    # calculte number of homes in each fold
    for fold_num in range(5):
        num_homes[fold_num] = gt[fold_num]['hvac'].reshape(-1, 1, 112,24).shape[0]
    homes = pd.Series(num_homes).sum()
    
    # for Constantly On appliances: hvac and fridge
    for appliance in ['hvac', 'fridge']:
        overall[appliance] = 0
        error[appliance] = {}
        for fold_num in range(5):
            error[appliance][fold_num] = mean_absolute_error(pred[fold_num][appliance].reshape(-1, 24), 
                                                             gt[fold_num][appliance].reshape(-1, 24))
            overall[appliance] += error[appliance][fold_num]*num_homes[fold_num]
        overall[appliance] /= homes
        
    # for ON/OFF appliances: dryer, dishwasher and microwave
    for appliance in ['dr', 'dw', 'mw']:
        error[appliance] = {}
        overall[appliance] = 0                                                                
        for fold_num in range(5):
            error[appliance][fold_num] = onoff_error(pred[fold_num][appliance].reshape(-1, 24), 
                                                     gt[fold_num][appliance].reshape(-1, 24), threshold[appliance])
            overall[appliance] += error[appliance][fold_num]*num_homes[fold_num]
        overall[appliance] /= homes
    
    
    return error, overall

In [30]:
pd.Series(calculate_error(cnn_tree_pred, test_gt, threshold)[1])

dr        695.579196
dw        218.650229
fridge     36.066408
hvac      397.938498
mw         23.917026
dtype: float64

In [6]:
dataset = 5
gid = 0
srv = 1
c = 0
cmd = {}
valid_error= {}
num_iterations = 20000
lr = 0.01
p = 0
for fold_num in range(5):
    valid_error[fold_num] = {}
    for lr in [0.01]:
        valid_error[fold_num][lr] = {}
        for order in list(itertools.permutations(['hvac', 'fridge', 'dr', 'dw', 'mw', 'residual'])):
#             print(fold_num, lr, order)
#             if order[0] == 'hvac':
#                 continue
            
#             valid_error[fold_num][lr][order] = {}


            o = "\', \'".join(str(x) for x in order)
            directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num, lr)
            filename = "valid-error-[\'{}\'].npy".format(o)

            full_path = directory + filename
            my_file = Path(full_path)
            if not my_file.exists():
                o = " ".join(str(x) for x in order)
                line = ("CUDA_VISIBLE_DEVICES={} python cnn-tree.py 3 {} {} 0 {} {} &".format(gid, lr, num_iterations, fold_num, o))
#                 print(line)

                c += 1
                if c%1 == 0:
                    gid+=1
                if gid == 4:
                    gid = 0
                    srv += 1
            else:
                valid_error[fold_num][lr][order] = np.load(full_path).item()

print(c)                               

0


In [15]:
results = {}
iters = {}
best_param = {}
for fold_num in range(5):
    results[fold_num] = {}
    best_param[fold_num] = {}
for fold_num in range(5):
    
    best_error = np.inf
    for lr in [0.01]:
        for order in valid_error[fold_num][lr]:
            for iters in valid_error[fold_num][lr][order]:
                cur_error = pd.Series(valid_error[fold_num][lr][order][iters]).drop(['residual']).mean()
                if best_error > cur_error:
                    best_error = cur_error
                    best_param[fold_num]['iters'] = iters
                    best_param[fold_num]['order'] = order
                    best_param[fold_num]['lr'] = lr

In [16]:
best_error = {}
for fold_num in range(5):
    order = best_param[fold_num]['order']
    iters = best_param[fold_num]['iters']
    lr = best_param[fold_num]['lr']
    
    o = "\', \'".join(str(x) for x in order)
    directory = "../code/baseline/cnn-tree/{}/{}/{}/20000/0.0/".format(dataset, fold_num, lr)
    filename = "valid-error-[\'{}\'].npy".format(o)
    full_path = directory + filename
    
    best_error[fold_num] = np.load(full_path).item()[iters]

In [17]:
best_error[0]

{'dr': 62.402404557995432,
 'dw': 9.0537611668229214,
 'fridge': 30.827749422713065,
 'hvac': 319.75299064023017,
 'mw': 15.164341752110095,
 'residual': 368.93570354779337}

In [19]:
for fold_num in range(5):

    print(best_param[fold_num]['order'])

('dr', 'residual', 'dw', 'mw', 'hvac', 'fridge')
('hvac', 'dw', 'mw', 'fridge', 'residual', 'dr')
('hvac', 'fridge', 'mw', 'dw', 'dr', 'residual')
('residual', 'dr', 'dw', 'hvac', 'mw', 'fridge')
('residual', 'dr', 'mw', 'dw', 'hvac', 'fridge')
